## Hate Intensity Reduction (HIP): Generation

Given a hateful sentence generate/paraphase it to in form of a normlised sentence. This normalised sentence is then rewarded with a reduction in intensity.

In [1]:
# !pip install numpy==1.19.5
# !pip install tensorflow==2.2.0
# !pip install transformers==3.4.0
# !pip install tensorflow-addons==0.10.0
# !pip install torch==1.4.0
# !pip install simpletransformers==0.49.0
# !pip install sklearn scipy

import tensorflow as tf
print(tf.__version__)

import transformers
import simpletransformers
print(transformers.__version__)

import numpy as np
print(np.__version__)

2.2.0


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


3.4.0
1.19.5


In [2]:
## The folder is setup to from google drive. If used else only the following lines needs commenting

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import random
import pickle 
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer
from transformers import DistilBertTokenizer, RobertaTokenizer, BertConfig, TFBertModel
import random
import sys 
import math

from simpletransformers.config.global_args import global_args
from simpletransformers.config.model_args import Seq2SeqArgs
from simpletransformers.seq2seq.seq2seq_utils import Seq2SeqDataset, SimpleSummarizationDataset

import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, load_model
from sklearn.metrics import mean_squared_error

import os

import logging
import os
import pickle
from multiprocessing import Pool
from typing import Tuple

import pandas as pd
import torch
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from torch.utils.data import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizer

import json
import logging
import math
import os
import random
import warnings
from dataclasses import asdict
from multiprocessing import Pool, cpu_count
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.auto import tqdm, trange

In [4]:
BASE_FOLDER = "/content/drive/MyDrive/hate_norm_kdd22/"
OUTPUT_FOLDER = "BART_GEN"
OUTPUT_FILE = "BART_GEN_FILE"
INPUT_HT_FOLDER="hate_intensity_linear_weights_att/"
INPUT_HT_FILE="hate_int_linear_trans42_ATT"
BERT_MODEL = "distilbert-base-uncased"
BART_INPUT = "train_df.csv"
MAX_LENGTH = 128
SEED = 42

USE_ATT = True

BERT_DROPOUT = 0.2
LSTM_UNITS = 50
DENSE_UNITS = 50
LSTM_DROPOUT = 0.1
DENSE_DROPOUT = 0.2


def random_seed(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

random_seed(SEED)

### Load the intensity module

In [5]:
tokenizer_intensity = DistilBertTokenizer.from_pretrained(
    BERT_MODEL,
    do_lower_case=True,
    add_special_tokens=True,
    max_length=MAX_LENGTH,
    pad_to_max_length=True)


def tokenize_bert(sentences, tokenizer=tokenizer_intensity):
    input_ids, input_masks, input_segments = [], [], []
    for sentence in tqdm(sentences):
        inputs = tokenizer_intensity.encode_plus(sentence,
                                                 add_special_tokens=True,
                                                 max_length=MAX_LENGTH,
                                                 pad_to_max_length=True,
                                                 return_attention_mask=True,
                                                 return_token_type_ids=True)
        # del inputs["special_tokens_mask"]
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])
    return np.asarray(input_ids, dtype='int32'), np.asarray(
        input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


config = BertConfig(dropout=BERT_DROPOUT,
                    attention_dropout=BERT_DROPOUT,
                    output_attentions=True)
config.output_hidden_states = False
transformer_model = TFBertModel.from_pretrained(BERT_MODEL, config=config)
for layer in transformer_model.layers[:3]:
    layer.trainable = False

input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH, ),
                                     name='input_token',
                                     dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH, ),
                                       name='masked_token',
                                       dtype='int32')
embedding_layer = transformer_model(input_ids_in,
                                    attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(LSTM_UNITS,
                         return_sequences=True,
                         dropout=LSTM_DROPOUT,
                         recurrent_dropout=LSTM_DROPOUT,
                         kernel_initializer='normal'))(embedding_layer)
if USE_ATT:
    X = tf.keras.layers.Attention(use_scale=True)([X, X])  # Use attention.
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(DENSE_UNITS,
                          activation='relu',
                          kernel_initializer='normal')(X)
X = tf.keras.layers.Dropout(DENSE_DROPOUT)(X)
X = tf.keras.layers.Dense(1, activation='linear',
                          kernel_initializer='normal')(X)
intensity_model = tf.keras.Model(inputs=[input_ids_in, input_masks_in],
                                 outputs=X)
intensity_model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['acc', tf.keras.metrics.RootMeanSquaredError()])
intensity_model.summary()
intensity_model.load_weights(BASE_FOLDER + INPUT_HT_FOLDER + INPUT_HT_FILE)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

2022-08-03 07:56:20.904693: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-08-03 07:56:20.920439: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-03 07:56:20.920534: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b5fa7e37f2cf): /proc/driver/nvidia/version does not exist
2022-08-03 07:56:20.925928: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-08-03 07:56:20.956422: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2199995000 Hz
2022-08-03 07:56:20.966271: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7d2d100 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-03 07:56

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 128, 768), ( 109482240   input_token[0][0]                
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 128, 100)     327600      tf_bert_model[0][0]              
______________________________________________________________________________________________

2022-08-03 07:56:51.622508: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 93763584 exceeds 10% of free system memory.
2022-08-03 07:57:00.704159: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 93763584 exceeds 10% of free system memory.


In [ ]:
## We copy the origina code of Seq2seq model so as to form a change in its loss function

In [6]:
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    BartConfig,
    BartForConditionalGeneration,
    BartTokenizer,
    BertConfig,
    BertForMaskedLM,
    BertModel,
    BertTokenizer,
    CamembertConfig,
    CamembertModel,
    CamembertTokenizer,
    DistilBertConfig,
    DistilBertModel,
    DistilBertTokenizer,
    ElectraConfig,
    ElectraModel,
    ElectraTokenizer,
    EncoderDecoderConfig,
    EncoderDecoderModel,
    LongformerConfig,
    LongformerModel,
    LongformerTokenizer,
    MarianConfig,
    MarianMTModel,
    MarianTokenizer,
    MobileBertConfig,
    MobileBertModel,
    MobileBertTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    RobertaConfig,
    RobertaModel,
    RobertaTokenizer,
    get_linear_schedule_with_warmup,
)

kfold = KFold(n_splits=2)

MODEL_CLASSES = {
    "auto": (AutoConfig, AutoModel, AutoTokenizer),
    "bart": (BartConfig, BartForConditionalGeneration, BartTokenizer),
    "bert": (BertConfig, BertModel, BertTokenizer),
    "camembert": (CamembertConfig, CamembertModel, CamembertTokenizer),
    "distilbert": (DistilBertConfig, DistilBertModel, DistilBertTokenizer),
    "electra": (ElectraConfig, ElectraModel, ElectraTokenizer),
    "longformer": (LongformerConfig, LongformerModel, LongformerTokenizer),
    "mobilebert": (MobileBertConfig, MobileBertModel, MobileBertTokenizer),
    "marian": (MarianConfig, MarianMTModel, MarianTokenizer),
    "roberta": (RobertaConfig, RobertaModel, RobertaTokenizer),
}

In [7]:
import csv
output = open('/content/drive/MyDrive/hate_norm_kdd22/pred_final_new7.csv',mode='w')
output_writer = csv.writer(output, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

heading=['Original_text','Truth','Pred1']
output_writer.writerow(heading)

27

In [ ]:
## In order to load run the 

In [10]:
class Seq2SeqModel:
    def __init__(
        self,
        encoder_type=None,
        encoder_name=None,
        decoder_name=None,
        encoder_decoder_type=None,
        encoder_decoder_name=None,
        config=None,
        args=None,
        use_cuda=True,
        cuda_device=-1,
        **kwargs,
    ):

        """
        Initializes a Seq2SeqModel.
        Args:
            encoder_type (optional): The type of model to use as the encoder.
            encoder_name (optional): The exact architecture and trained weights to use. This may be a Hugging Face Transformers compatible pre-trained model, a community model, or the path to a directory containing model files.
            decoder_name (optional): The exact architecture and trained weights to use. This may be a Hugging Face Transformers compatible pre-trained model, a community model, or the path to a directory containing model files.
                                    Must be the same "size" as the encoder model (base/base, large/large, etc.)
            encoder_decoder_type (optional): The type of encoder-decoder model. (E.g. bart)
            encoder_decoder_name (optional): The path to a directory containing the saved encoder and decoder of a Seq2SeqModel. (E.g. "outputs/") OR a valid BART or MarianMT model.
            config (optional): A configuration file to build an EncoderDecoderModel.
            args (optional): Default args will be used if this parameter is not provided. If provided, it should be a dict containing the args that should be changed in the default args.
            use_cuda (optional): Use GPU if available. Setting to False will force model to use CPU only.
            cuda_device (optional): Specific GPU that should be used. Will use the first available GPU by default.
            **kwargs (optional): For providing proxies, force_download, resume_download, cache_dir and other options specific to the 'from_pretrained' implementation where this will be supplied.
        """  # noqa: ignore flake8"

        if not config:
            # if not ((encoder_name and decoder_name) or encoder_decoder_name) and not encoder_type:
            if not ((encoder_name and decoder_name) or encoder_decoder_name):
                raise ValueError(
                    "You must specify a Seq2Seq config \t OR \t"
                    "encoder_type, encoder_name, and decoder_name OR \t \t"
                    "encoder_type and encoder_decoder_name")
            elif not (encoder_type or encoder_decoder_type):
                raise ValueError(
                    "You must specify a Seq2Seq config \t OR \t"
                    "encoder_type, encoder_name, and decoder_name \t OR \t"
                    "encoder_type and encoder_decoder_name")

        self.args = self._load_model_args(encoder_decoder_name)

        if isinstance(args, dict):
            self.args.update_from_dict(args)
        elif isinstance(args, Seq2SeqArgs):
            self.args = args

        if "sweep_config" in kwargs:
            sweep_config = kwargs.pop("sweep_config")
            sweep_values = {
                key: value["value"]
                for key, value in sweep_config.as_dict().items()
                if key != "_wandb"
            }
            self.args.update_from_dict(sweep_values)

        if self.args.manual_seed:
            random.seed(self.args.manual_seed)
            np.random.seed(self.args.manual_seed)
            torch.manual_seed(self.args.manual_seed)
            if self.args.n_gpu > 0:
                torch.cuda.manual_seed_all(self.args.manual_seed)

        if use_cuda:
            if torch.cuda.is_available():
                if cuda_device == -1:
                    self.device = torch.device("cuda")
                else:
                    self.device = torch.device(f"cuda:{cuda_device}")
            else:
                raise ValueError(
                    "'use_cuda' set to True when cuda is unavailable."
                    "Make sure CUDA is available or set `use_cuda=False`.")
        else:
            self.device = "cpu"

        self.results = {}

        if not use_cuda:
            self.args.fp16 = False

        # config = EncoderDecoderConfig.from_encoder_decoder_configs(config, config)
        if encoder_decoder_type:
            config_class, model_class, tokenizer_class = MODEL_CLASSES[
                encoder_decoder_type]
        else:
            config_class, model_class, tokenizer_class = MODEL_CLASSES[
                encoder_type]

        if encoder_decoder_type in ["bart", "marian"]:
            self.model = model_class.from_pretrained(encoder_decoder_name)
            if encoder_decoder_type == "bart":
                self.encoder_tokenizer = tokenizer_class.from_pretrained(
                    encoder_decoder_name)
            elif encoder_decoder_type == "marian":
                if self.args.base_marian_model_name:
                    self.encoder_tokenizer = tokenizer_class.from_pretrained(
                        self.args.base_marian_model_name)
                else:
                    self.encoder_tokenizer = tokenizer_class.from_pretrained(
                        encoder_decoder_name)
            self.decoder_tokenizer = self.encoder_tokenizer
            self.config = self.model.config
        else:
            if encoder_decoder_name:
                # self.model = EncoderDecoderModel.from_pretrained(encoder_decoder_name)
                self.model = EncoderDecoderModel.from_encoder_decoder_pretrained(
                    os.path.join(encoder_decoder_name, "encoder"),
                    os.path.join(encoder_decoder_name, "decoder"))
                self.model.encoder = model_class.from_pretrained(
                    os.path.join(encoder_decoder_name, "encoder"))
                self.model.decoder = BertForMaskedLM.from_pretrained(
                    os.path.join(encoder_decoder_name, "decoder"))
                self.encoder_tokenizer = tokenizer_class.from_pretrained(
                    os.path.join(encoder_decoder_name, "encoder"))
                self.decoder_tokenizer = BertTokenizer.from_pretrained(
                    os.path.join(encoder_decoder_name, "decoder"))
            else:
                self.model = EncoderDecoderModel.from_encoder_decoder_pretrained(
                    encoder_name, decoder_name, config=config)
                self.encoder_tokenizer = tokenizer_class.from_pretrained(
                    encoder_name)
                self.decoder_tokenizer = BertTokenizer.from_pretrained(
                    decoder_name)
            self.encoder_config = self.model.config.encoder
            self.decoder_config = self.model.config.decoder

        # if self.args.wandb_project and not wandb_available:
        #     warnings.warn("wandb_project specified but wandb is not available. Wandb disabled.")
        #     self.args.wandb_project = None

        if encoder_decoder_name:
            self.args.model_name = encoder_decoder_name

            # # Checking if we are loading from a saved model or using a pre-trained model
            # if not saved_model_args and encoder_decoder_type == "marian":
            # Need to store base pre-trained model name to get the tokenizer when loading a saved model
            self.args.base_marian_model_name = encoder_decoder_name

        elif encoder_name and decoder_name:
            self.args.model_name = encoder_name + "-" + decoder_name
        else:
            self.args.model_name = "encoder-decoder"

        if encoder_decoder_type:
            self.args.model_type = encoder_decoder_type
        elif encoder_type:
            self.args.model_type = encoder_type + "-bert"
        else:
            self.args.model_type = "encoder-decoder"

    def train_model(
        self,
        train_data,
        model_lr,
        tokenizer_new,
        labels,
        test_data,
        output_dir=None,
        show_running_loss=True,
        args=None,
        eval_data=None,
        verbose=True,
        **kwargs,
    ):
        """
        Trains the model using 'train_data'
        Args:
            train_data: Pandas DataFrame containing the 2 columns - `input_text`, `target_text`.
                        - `input_text`: The input text sequence.
                        - `target_text`: The target text sequence
            output_dir: The directory where model files will be saved. If not given, self.args.output_dir will be used.
            show_running_loss (optional): Set to False to prevent running loss from being printed to console. Defaults to True.
            args (optional): Optional changes to the args dict of the model. Any changes made will persist for the model.
            eval_data (optional): A DataFrame against which evaluation will be performed when evaluate_during_training is enabled. Is required if evaluate_during_training is enabled.
            **kwargs: Additional metrics that should be used. Pass in the metrics as keyword arguments (name of metric: function to use).
                        A metric function should take in two parameters. The first parameter will be the true labels, and the second parameter will be the predictions. Both inputs
                        will be lists of strings. Note that this will slow down training significantly as the predicted sequences need to be generated.
        Returns:
            None
        """  # noqa: ignore flake8"
        print("in train_model**** eval_data", eval_data)
        if args:
            self.args.update_from_dict(args)

        # if self.args.silent:
        #     show_running_loss = False
        '''if self.args.evaluate_during_training and eval_data is None:
            raise ValueError(
                "evaluate_during_training is enabled but eval_data is not specified."
                " Pass eval_data to model.train_model() if using evaluate_during_training."
            )'''

        if not output_dir:
            output_dir = self.args.output_dir

        if os.path.exists(output_dir) and os.listdir(
                output_dir) and not self.args.overwrite_output_dir:
            raise ValueError(
                "Output directory ({}) already exists and is not empty."
                " Set args.overwrite_output_dir = True to overcome.".format(
                    output_dir))

        self._move_model_to_device()
        os.makedirs(output_dir, exist_ok=True)
        print("train")
        print(len(train_data))
        print(train_data)
        print("labels: ", labels)
        print("type: ", type(labels))
        print("Beginiing 1st kfold")
        train_dataset = self.load_and_cache_examples(train_data,
                                                     verbose=verbose)
        # tr_labels=labels['id'].to_list()
        tr_labels = labels.tolist()
        print(type(tr_labels))
        # global_step, tr_loss = self.train(train_dataset,output_dir,model_lr,tr_labels,tokenizer_new,show_running_loss=show_running_loss,eval_data=train_df.iloc[eval_index],verbose=verbose,**kwargs,)

        # IMPORTANT -------------------------------- FIX EVAL INDEX ^^
        global_step, tr_loss = self.train(
            train_dataset,
            output_dir,
            model_lr,
            tr_labels,
            tokenizer_new,
            show_running_loss=show_running_loss,
            eval_data=eval_data,
            verbose=verbose,
            **kwargs,
        )
        # self._save_model(self.args.output_dir, model=self.model)
        # output_eval_file = os.path.join(output_dir, "eval_results.txt")
        # with open(output_eval_file, "w") as writer:
        #   for key in sorted(results.keys()):
        #     writer.write("{} = {}\n".format(key, str(results[key])))
        to_predict = [
            str(input_text) for input_text in test_df["input_text"].tolist()
        ]
        preds = model.predict(to_predict)
        # pred = model(to_predict)
        wr = []
        c = "  "
        w = "Crosfold begin"
        # output_writer = open("output.txt","wb")
        for i, text in enumerate(test_df["input_text"].tolist()):
            row = []
            row.append(str(text))
            row.append(test_df["target_text"][i])
            wr.append(w)
            wr.append(c)
            row.append(str(preds[i]))
            output_writer.writerow(row)
        output_writer.writerow(wr)

        # model_to_save = self.model.module if hasattr(self.model, "module") else self.model
        # model_to_save.save_pretrained(output_dir)
        # self.encoder_tokenizer.save_pretrained(output_dir)
        # self.decoder_tokenizer.save_pretrained(output_dir)
        # torch.save(self.args, os.path.join(output_dir, "training_args.bin"))

        if verbose:
            logger.info(" Training of {} model complete. Saved to {}.".format(
                self.args.model_name, output_dir))

    def train(
        self,
        train_dataset,
        output_dir,
        model_lr,
        tr_labels,
        tokenizer_new,
        show_running_loss=True,
        eval_data=None,
        verbose=True,
        **kwargs,
    ):
        """
        Trains the model on train_dataset.
        Utility function to be used by the train_model() method. Not intended to be used directly.
        """
        print("ln", len(tr_labels))
        # model = self.model
        args = self.args

        # tb_writer = SummaryWriter(logdir=args.tensorboard_dir)

        train_dataloader = DataLoader(
            train_dataset,
            batch_size=args.train_batch_size,
            num_workers=self.args.dataloader_num_workers,
        )
        train_dataloader2 = DataLoader(
            tr_labels,
            batch_size=args.train_batch_size,
            num_workers=self.args.dataloader_num_workers,
        )
        print("1 done")

        if args.max_steps > 0:
            t_total = args.max_steps
            args.num_train_epochs = args.max_steps // (
                len(train_dataloader) // args.gradient_accumulation_steps) + 1
        else:
            t_total = len(
                train_dataloader
            ) // args.gradient_accumulation_steps * args.num_train_epochs

        no_decay = ["bias", "LayerNorm.weight"]

        optimizer_grouped_parameters = []
        custom_parameter_names = set()
        for group in self.args.custom_parameter_groups:
            params = group.pop("params")
            custom_parameter_names.update(params)
            param_group = {**group}
            param_group["params"] = [
                p for n, p in self.model.named_parameters() if n in params
            ]
            optimizer_grouped_parameters.append(param_group)

        for group in self.args.custom_layer_parameters:
            layer_number = group.pop("layer")
            layer = f"layer.{layer_number}."
            group_d = {**group}
            group_nd = {**group}
            group_nd["weight_decay"] = 0.0
            params_d = []
            params_nd = []
            for n, p in self.model.named_parameters():
                if n not in custom_parameter_names and layer in n:
                    if any(nd in n for nd in no_decay):
                        params_nd.append(p)
                    else:
                        params_d.append(p)
                    custom_parameter_names.add(n)
            group_d["params"] = params_d
            group_nd["params"] = params_nd

            optimizer_grouped_parameters.append(group_d)
            optimizer_grouped_parameters.append(group_nd)

        if not self.args.train_custom_parameters_only:
            optimizer_grouped_parameters.extend([
                {
                    "params": [
                        p for n, p in self.model.named_parameters()
                        if n not in custom_parameter_names and not any(
                            nd in n for nd in no_decay)
                    ],
                    "weight_decay":
                    args.weight_decay,
                },
                {
                    "params": [
                        p for n, p in self.model.named_parameters()
                        if n not in custom_parameter_names and any(
                            nd in n for nd in no_decay)
                    ],
                    "weight_decay":
                    0.0,
                },
            ])

        warmup_steps = math.ceil(t_total * args.warmup_ratio)
        args.warmup_steps = warmup_steps if args.warmup_steps == 0 else args.warmup_steps

        # TODO: Use custom optimizer like with BertSum?
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=args.learning_rate,
                          eps=args.adam_epsilon)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=args.warmup_steps,
            num_training_steps=t_total)

        if (args.model_name and os.path.isfile(
                os.path.join(args.model_name, "optimizer.pt"))
                and os.path.isfile(
                    os.path.join(args.model_name, "scheduler.pt"))):
            # Load in optimizer and scheduler states
            optimizer.load_state_dict(
                torch.load(os.path.join(args.model_name, "optimizer.pt")))
            scheduler.load_state_dict(
                torch.load(os.path.join(args.model_name, "scheduler.pt")))

        if args.n_gpu > 1:
            self.model = torch.nn.DataParallel(model)

        logger.info(" Training started")

        global_step = 0
        tr_loss, logging_loss = 0.0, 0.0
        self.model.zero_grad()
        train_iterator = trange(int(args.num_train_epochs),
                                desc="Epoch",
                                disable=args.silent,
                                mininterval=0)
        epoch_number = 0
        best_eval_metric = None
        early_stopping_counter = 0
        steps_trained_in_current_epoch = 0
        epochs_trained = 0

        if args.model_name and os.path.exists(args.model_name):
            try:
                # set global_step to gobal_step of last saved checkpoint from model path
                checkpoint_suffix = args.model_name.split("/")[-1].split("-")
                if len(checkpoint_suffix) > 2:
                    checkpoint_suffix = checkpoint_suffix[1]
                else:
                    checkpoint_suffix = checkpoint_suffix[-1]
                global_step = int(checkpoint_suffix)
                epochs_trained = global_step // (
                    len(train_dataloader) // args.gradient_accumulation_steps)
                steps_trained_in_current_epoch = global_step % (
                    len(train_dataloader) // args.gradient_accumulation_steps)

                logger.info(
                    "   Continuing training from checkpoint, will skip to saved global_step"
                )
                logger.info("   Continuing training from epoch %d",
                            epochs_trained)
                logger.info("   Continuing training from global step %d",
                            global_step)
                logger.info(
                    "   Will skip the first %d steps in the current epoch",
                    steps_trained_in_current_epoch)
            except ValueError:
                logger.info("   Starting fine-tuning.")

        if args.evaluate_during_training:
            training_progress_scores = self._create_training_progress_scores(
                **kwargs)

        # if args.wandb_project:
        #     wandb.init(project=args.wandb_project, config={**asdict(args)}, **args.wandb_kwargs)
        #     wandb.watch(self.model)

        if args.fp16:
            from torch.cuda import amp

            scaler = amp.GradScaler()

        self.model.train()
        for current_epoch in train_iterator:
            if epochs_trained > 0:
                epochs_trained -= 1
                continue
            train_iterator.set_description(
                f"Epoch {epoch_number + 1} of {args.num_train_epochs}")
            batch_iterator = tqdm(
                train_dataloader,
                desc=f"Running Epoch {epoch_number} of {args.num_train_epochs}",
                disable=args.silent,
                mininterval=0,
            )
            batch_iterator2 = tqdm(
                train_dataloader2,
                disable=args.silent,
                mininterval=0,
            )
            print("2 done")
            count = 0
            for step, batch1 in enumerate(zip(batch_iterator,
                                              batch_iterator2)):
                print("here*********", steps_trained_in_current_epoch)
                if steps_trained_in_current_epoch > 0:
                    print("here", steps_trained_in_current_epoch)
                    steps_trained_in_current_epoch -= 1
                    continue
                # batch = tuple(t.to(device) for t in batch)
                print("range(len(batch1)", len(batch1), args.train_batch_size)
                # print("BATCH 1",batch1)
                # batch_p = [self.model.predict(np.array(X_te[i:i+args.train_batch_size]))[i] for i in range(len(batch1))]
                # batch_p = [self.model(np.array(X_te[i:i+args.train_batch_size]))[i] for i in range(len(batch1))]
                # batch = [np.argmax(batch_p[i], axis=-1) for i in range(len(batch_p))]
                batch = batch1
                print("batch: ", len(batch[0]))
                inputs = self._get_inputs_dict(batch[0])
                # print("batch: ",batch[1])
                print("inputs")
                print(len(inputs['input_ids']))
                print(inputs.keys())
                count = count + 1
                print(
                    "-----------------------------------------------------------"
                )
                if args.fp16:
                    with amp.autocast():
                        outputs = model(**inputs)
                        # model outputs are always tuple in pytorch-transformers (see doc)
                        loss = outputs[0]
                else:
                    outputs = self.model(**inputs)
                    # model outputs are always tuple in pytorch-transformers (see doc)
                    print("outputs", outputs[2].shape)
                    loss = outputs[0]
                    print("training", loss)

                outputs1 = self.model.generate(
                    input_ids=inputs['input_ids'],
                    num_beams=self.args.num_beams,
                    max_length=self.args.max_length,
                    length_penalty=self.args.length_penalty,
                    early_stopping=self.args.early_stopping,
                    repetition_penalty=self.args.repetition_penalty,
                    do_sample=self.args.do_sample,
                    top_k=self.args.top_k,
                    top_p=self.args.top_p,
                    num_return_sequences=self.args.num_return_sequences,
                )
                outputs_1 = [
                    self.decoder_tokenizer.decode(
                        output_id,
                        skip_special_tokens=True,
                        clean_up_tokenization_spaces=True)
                    for output_id in outputs1
                ]
                
                ## IMP NOTE: Introducting our loss.
                print(type(outputs_1))
                print(outputs_1)
                a, b, c = tokenize_bert(outputs_1, tokenizer_new)
                print("a: ", len(a))
                print("b: ", len(b))
                out = model_lr.predict([a, b])
                pred_r = [rd[0] for rd in out]
                true = batch[1].tolist()
                print("out: ", pred_r)
                print("bathc: ", batch[1].tolist())
                los1 = mean_squared_error(true, pred_r)
                print("2nd loss: ", los1)
                if args.n_gpu > 1:
                    loss = loss.mean(
                    )  # mean() to average on multi-gpu parallel training

                current_loss = loss.item()
                lo = torch.tensor(los1) + loss
                print("lo:", lo)
                t = los1 + loss
                print("old loss: ", loss)
                loss = loss + los1
                print("new loss: ", loss)
                print("t: ", t)
                if show_running_loss:
                    batch_iterator.set_description(
                        f"Epochs {epoch_number}/{args.num_train_epochs}. Running Loss: {current_loss:9.4f}"
                    )

                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps

                if args.fp16:
                    scaler.scale(loss).backward()
                else:
                    loss.backward()

                tr_loss += loss.item()
                if (step + 1) % args.gradient_accumulation_steps == 0:
                    if args.fp16:
                        scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(),
                                                   args.max_grad_norm)

                    if args.fp16:
                        scaler.step(optimizer)
                        scaler.update()
                    else:
                        optimizer.step()
                    scheduler.step()  # Update learning rate schedule
                    self.model.zero_grad()
                    global_step += 1

                    if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                        # Log metrics
                        # tb_writer.add_scalar("lr",
                        #                      scheduler.get_lr()[0],
                        # #                      global_step)
                        # tb_writer.add_scalar("loss", (tr_loss - logging_loss) /
                        #                      args.logging_steps, global_step)
                        logging_loss = tr_loss
                        # if args.wandb_project:
                        #     wandb.log(
                        #         {
                        #             "Training loss": current_loss,
                        #             "lr": scheduler.get_lr()[0],
                        #             "global_step": global_step,
                        #         }
                        #     )

                    if args.save_steps > 0: 
                    # and global_step % args.save_steps == 0:
                        # Save model checkpoint
                        output_dir_current = os.path.join(
                            output_dir, "checkpoint-{}".format(global_step))

                        self._save_model(output_dir_current,
                                         optimizer,
                                         scheduler,
                                         model=model)  # SAVE COMMENTED

                    if args.evaluate_during_training and (
                            args.evaluate_during_training_steps > 0
                            and global_step %
                            args.evaluate_during_training_steps == 0):
                        # Only evaluate when single GPU otherwise metrics may not average well
                        results = self.eval_model(
                            eval_data,
                            verbose=verbose
                            and args.evaluate_during_training_verbose,
                            silent=args.evaluate_during_training_silent,
                            **kwargs,
                        )
                        for key, value in results.items():
                          pass
                            # tb_writer.add_scalar("eval_{}".format(key), value,
                            #                      global_step)

                        output_dir_current = os.path.join(
                            output_dir, "checkpoint-{}".format(global_step))

                        if args.save_eval_checkpoints:
                            self._save_model(output_dir_current,
                                             optimizer,
                                             scheduler,
                                             model=model,
                                             results=results)

                        training_progress_scores["global_step"].append(
                            global_step)
                        training_progress_scores["train_loss"].append(
                            current_loss)
                        for key in results:
                            training_progress_scores[key].append(results[key])
                        report = pd.DataFrame(training_progress_scores)
                        report.to_csv(
                            os.path.join(args.output_dir,
                                         "training_progress_scores.csv"),
                            index=False,
                        )

                        # if args.wandb_project:
                        #     wandb.log(self._get_last_metrics(training_progress_scores))

                        if not best_eval_metric:
                            best_eval_metric = results[
                                args.early_stopping_metric]
                            if args.save_best_model:
                                self._save_model(args.best_model_dir,
                                                 optimizer,
                                                 scheduler,
                                                 model=model,
                                                 results=results)
                        if best_eval_metric and args.early_stopping_metric_minimize:
                            if results[
                                    args.
                                    early_stopping_metric] - best_eval_metric < args.early_stopping_delta:
                                best_eval_metric = results[
                                    args.early_stopping_metric]
                                if args.save_best_model:
                                    self._save_model(args.best_model_dir,
                                                     optimizer,
                                                     scheduler,
                                                     model=model,
                                                     results=results)
                                early_stopping_counter = 0
                            else:
                                if args.use_early_stopping:
                                    if early_stopping_counter < args.early_stopping_patience:
                                        early_stopping_counter += 1
                                        if verbose:
                                            logger.info(
                                                f" No improvement in {args.early_stopping_metric}"
                                            )
                                            logger.info(
                                                f" Current step: {early_stopping_counter}"
                                            )
                                            logger.info(
                                                f" Early stopping patience: {args.early_stopping_patience}"
                                            )
                                    else:
                                        if verbose:
                                            logger.info(
                                                f" Patience of {args.early_stopping_patience} steps reached"
                                            )
                                            logger.info(
                                                " Training terminated.")
                                            train_iterator.close()
                                        return global_step, tr_loss / global_step
                        else:
                            if results[
                                    args.
                                    early_stopping_metric] - best_eval_metric > args.early_stopping_delta:
                                best_eval_metric = results[
                                    args.early_stopping_metric]
                                if args.save_best_model:
                                    self._save_model(args.best_model_dir,
                                                     optimizer,
                                                     scheduler,
                                                     model=model,
                                                     results=results)
                                early_stopping_counter = 0
                            else:
                                if args.use_early_stopping:
                                    if early_stopping_counter < args.early_stopping_patience:
                                        early_stopping_counter += 1
                                        if verbose:
                                            logger.info(
                                                f" No improvement in {args.early_stopping_metric}"
                                            )
                                            logger.info(
                                                f" Current step: {early_stopping_counter}"
                                            )
                                            logger.info(
                                                f" Early stopping patience: {args.early_stopping_patience}"
                                            )
                                    else:
                                        if verbose:
                                            logger.info(
                                                f" Patience of {args.early_stopping_patience} steps reached"
                                            )
                                            logger.info(
                                                " Training terminated.")
                                            train_iterator.close()
                                        return global_step, tr_loss / global_step

            print(count)
            epoch_number += 1
            output_dir_current = os.path.join(
                output_dir,
                "checkpoint-{}-epoch-{}".format(global_step, epoch_number))

            if args.save_model_every_epoch or args.evaluate_during_training:
                os.makedirs(output_dir_current, exist_ok=True)

            if args.save_model_every_epoch:
                # self._save_model(output_dir_current, optimizer, scheduler, model=model) # SAVE
                pass
            if args.evaluate_during_training:
                results = self.eval_model(
                    eval_data,
                    verbose=verbose and args.evaluate_during_training_verbose,
                    silent=args.evaluate_during_training_silent,
                    **kwargs,
                )

                if args.save_eval_checkpoints:
                    self._save_model(output_dir_current,
                                     optimizer,
                                     scheduler,
                                     results=results)

                training_progress_scores["global_step"].append(global_step)
                training_progress_scores["train_loss"].append(current_loss)
                for key in results:
                    training_progress_scores[key].append(results[key])
                report = pd.DataFrame(training_progress_scores)
                report.to_csv(os.path.join(args.output_dir,
                                           "training_progress_scores.csv"),
                              index=False)

                # if args.wandb_project:
                #     wandb.log(self._get_last_metrics(training_progress_scores))

                if not best_eval_metric:
                    best_eval_metric = results[args.early_stopping_metric]
                    if args.save_best_model:
                        pass
                        # self._save_model(args.best_model_dir, optimizer, scheduler, model=model, results=results)
                if best_eval_metric and args.early_stopping_metric_minimize:
                    if results[
                            args.
                            early_stopping_metric] - best_eval_metric < args.early_stopping_delta:
                        best_eval_metric = results[args.early_stopping_metric]
                        if args.save_best_model:
                            # pass
                            self._save_model(args.best_model_dir,
                                             optimizer,
                                             scheduler,
                                             model=model,
                                             results=results)  # SAVE COMMENTED
                        early_stopping_counter = 0
                    else:
                        if args.use_early_stopping and args.early_stopping_consider_epochs:
                            if early_stopping_counter < args.early_stopping_patience:
                                early_stopping_counter += 1
                                if verbose:
                                    logger.info(
                                        f" No improvement in {args.early_stopping_metric}"
                                    )
                                    logger.info(
                                        f" Current step: {early_stopping_counter}"
                                    )
                                    logger.info(
                                        f" Early stopping patience: {args.early_stopping_patience}"
                                    )
                            else:
                                if verbose:
                                    logger.info(
                                        f" Patience of {args.early_stopping_patience} steps reached"
                                    )
                                    logger.info(" Training terminated.")
                                    train_iterator.close()
                                return global_step, tr_loss / global_step
                else:
                    if results[
                            args.
                            early_stopping_metric] - best_eval_metric > args.early_stopping_delta:
                        best_eval_metric = results[args.early_stopping_metric]
                        if args.save_best_model:
                            # pass
                            self._save_model(args.best_model_dir,
                                             optimizer,
                                             scheduler,
                                             model=model,
                                             results=results)  # SAVE COMMENTED
                        early_stopping_counter = 0
                    else:
                        if args.use_early_stopping and args.early_stopping_consider_epochs:
                            if early_stopping_counter < args.early_stopping_patience:
                                early_stopping_counter += 1
                                if verbose:
                                    logger.info(
                                        f" No improvement in {args.early_stopping_metric}"
                                    )
                                    logger.info(
                                        f" Current step: {early_stopping_counter}"
                                    )
                                    logger.info(
                                        f" Early stopping patience: {args.early_stopping_patience}"
                                    )
                            else:
                                if verbose:
                                    logger.info(
                                        f" Patience of {args.early_stopping_patience} steps reached"
                                    )
                                    logger.info(" Training terminated.")
                                    train_iterator.close()
                                return global_step, tr_loss / global_step
        print("complete first kfold")
        return global_step, tr_loss / global_step

    def eval_model(self,
                   eval_data,
                   output_dir=None,
                   verbose=True,
                   silent=False,
                   **kwargs):
        """
        Evaluates the model on eval_data. Saves results to output_dir.
        Args:
            eval_data: Pandas DataFrame containing the 2 columns - `input_text`, `target_text`.
                        - `input_text`: The input text sequence.
                        - `target_text`: The target text sequence.
            output_dir: The directory where model files will be saved. If not given, self.args.output_dir will be used.
            verbose: If verbose, results will be printed to the console on completion of evaluation.
            silent: If silent, tqdm progress bars will be hidden.
            **kwargs: Additional metrics that should be used. Pass in the metrics as keyword arguments (name of metric: function to use).
                        A metric function should take in two parameters. The first parameter will be the true labels, and the second parameter will be the predictions. Both inputs
                        will be lists of strings. Note that this will slow down evaluation significantly as the predicted sequences need to be generated.
        Returns:
            results: Dictionary containing evaluation results.
        """  # noqa: ignore flake8"

        if not output_dir:
            output_dir = self.args.output_dir
        print("eval_data**********",eval_data)
        self._move_model_to_device()

        eval_dataset = self.load_and_cache_examples(eval_data,
                                                    evaluate=True,
                                                    verbose=verbose,
                                                    silent=silent)
        os.makedirs(output_dir, exist_ok=True)

        result = self.evaluate(eval_dataset,
                               output_dir,
                               verbose=verbose,
                               silent=silent,
                               **kwargs)
        self.results.update(result)

        if self.args.evaluate_generated_text:
            to_predict = eval_data["input_text"].tolist()
            preds = self.predict(to_predict)

            result = self.compute_metrics(eval_data["target_text"].tolist(),
                                          preds, **kwargs)
            self.results.update(result)

        if verbose:
            logger.info(self.results)

        return self.results

    def evaluate(self,
                 eval_dataset,
                 output_dir,
                 verbose=True,
                 silent=False,
                 **kwargs):
        """
        Evaluates the model on eval_dataset.
        Utility function to be used by the eval_model() method. Not intended to be used directly.
        """

        model = self.model
        args = self.args
        eval_output_dir = output_dir

        results = {}

        eval_sampler = SequentialSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset,
                                     sampler=eval_sampler,
                                     batch_size=args.eval_batch_size)

        if args.n_gpu > 1:
            model = torch.nn.DataParallel(model)

        eval_loss = 0.0
        nb_eval_steps = 0
        model.eval()

        for batch in tqdm(eval_dataloader,
                          disable=args.silent or silent,
                          desc="Running Evaluation"):
            # batch = tuple(t.to(device) for t in batch)

            inputs = self._get_inputs_dict(batch)
            with torch.no_grad():
                outputs = model(**inputs)
                loss = outputs[0]
                print("Evaluation loss", loss)
                eval_loss += loss.mean().item()
            nb_eval_steps += 1

        eval_loss = eval_loss / nb_eval_steps

        results["eval_loss"] = eval_loss

        output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            for key in sorted(results.keys()):
                writer.write("{} = {}\n".format(key, str(results[key])))

        return results

    def predict(self, to_predict):
        """
        Performs predictions on a list of text.
        Args:
            to_predict: A python list of text (str) to be sent to the model for prediction. Note that the prefix should be prepended to the text.
        Returns:
            preds: A python list of the generated sequences.
        """  # noqa: ignore flake8"

        self._move_model_to_device()

        all_outputs = []
        # Batching
        for batch in [
                to_predict[i:i + self.args.eval_batch_size]
                for i in range(0, len(to_predict), self.args.eval_batch_size)
        ]:
            if self.args.model_type == "marian":
                input_ids = self.encoder_tokenizer.prepare_translation_batch(
                    batch,
                    max_length=self.args.max_seq_length,
                    pad_to_max_length=True,
                    return_tensors="pt",
                )["input_ids"]
            else:
                input_ids = self.encoder_tokenizer.batch_encode_plus(
                    batch,
                    max_length=self.args.max_seq_length,
                    pad_to_max_length=True,
                    return_tensors="pt",
                )["input_ids"]
            # del input_ids["special_tokens_mask"]
            input_ids = input_ids.to(self.device)

            if self.args.model_type in ["bart", "marian"]:
                outputs = self.model.generate(
                    input_ids=input_ids,
                    num_beams=self.args.num_beams,
                    max_length=self.args.max_length,
                    length_penalty=self.args.length_penalty,
                    early_stopping=self.args.early_stopping,
                    repetition_penalty=self.args.repetition_penalty,
                    do_sample=self.args.do_sample,
                    top_k=self.args.top_k,
                    top_p=self.args.top_p,
                    num_return_sequences=self.args.num_return_sequences,
                )
            else:
                outputs = self.model.generate(
                    input_ids=input_ids,
                    decoder_start_token_id=self.model.config.decoder.
                    pad_token_id,
                    num_beams=self.args.num_beams,
                    max_length=self.args.max_length,
                    length_penalty=self.args.length_penalty,
                    early_stopping=self.args.early_stopping,
                    repetition_penalty=self.args.repetition_penalty,
                    do_sample=self.args.do_sample,
                    top_k=self.args.top_k,
                    top_p=self.args.top_p,
                    num_return_sequences=self.args.num_return_sequences,
                )

            all_outputs.extend(outputs.cpu().numpy())

        if self.args.use_multiprocessed_decoding:
            self.model.to("cpu")
            with Pool(self.args.process_count) as p:
                outputs = list(
                    tqdm(
                        p.imap(self._decode,
                               all_outputs,
                               chunksize=self.args.multiprocessing_chunksize),
                        total=len(all_outputs),
                        desc="Decoding outputs",
                        disable=self.args.silent,
                    ))
            self._move_model_to_device()
        else:
            outputs = [
                self.decoder_tokenizer.decode(
                    output_id,
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=True)
                for output_id in all_outputs
            ]

        if self.args.num_return_sequences > 1:
            return [
                outputs[i:i + self.args.num_return_sequences]
                for i in range(0, len(outputs), self.args.num_return_sequences)
            ]
        else:
            return outputs

    def _decode(self, output_id):
        return self.decoder_tokenizer.decode(output_id,
                                             skip_special_tokens=True,
                                             clean_up_tokenization_spaces=True)

    def compute_metrics(self, labels, preds, **kwargs):
        """
        Computes the evaluation metrics for the model predictions.
        Args:
            labels: List of target sequences
            preds: List of model generated outputs
            **kwargs: Custom metrics that should be used. Pass in the metrics as keyword arguments (name of metric: function to use).
                        A metric function should take in two parameters. The first parameter will be the true labels, and the second parameter will be the predictions. Both inputs
                        will be lists of strings. Note that this will slow down evaluation significantly as the predicted sequences need to be generated.
        Returns:
            result: Dictionary containing evaluation results.
        """  # noqa: ignore flake8"
        # assert len(labels) == len(preds)

        results = {}
        for metric, func in kwargs.items():
            results[metric] = func(labels, preds)

        return results

    def load_and_cache_examples(self,
                                data,
                                evaluate=False,
                                no_cache=False,
                                verbose=True,
                                silent=False):
        """
        Creates a T5Dataset from data.
        Utility function for train() and eval() methods. Not intended to be used directly.
        """

        encoder_tokenizer = self.encoder_tokenizer
        decoder_tokenizer = self.decoder_tokenizer
        args = self.args
        print("data*********************",data)
        if not no_cache:
            no_cache = args.no_cache

        if not no_cache:
            os.makedirs(self.args.cache_dir, exist_ok=True)

        mode = "dev" if evaluate else "train"

        if args.dataset_class:
            CustomDataset = args.dataset_class
            return CustomDataset(encoder_tokenizer, decoder_tokenizer, args,
                                 data, mode)
        else:
            if args.model_type in ["bart", "marian"]:
                return SimpleSummarizationDataset(encoder_tokenizer, self.args,
                                                  data, mode)
            else:
                return Seq2SeqDataset(
                    encoder_tokenizer,
                    decoder_tokenizer,
                    self.args,
                    data,
                    mode,
                )

    def _create_training_progress_scores(self, **kwargs):
        extra_metrics = {key: [] for key in kwargs}
        training_progress_scores = {
            "global_step": [],
            "eval_loss": [],
            "train_loss": [],
            **extra_metrics,
        }

        return training_progress_scores

    def _get_last_metrics(self, metric_values):
        return {metric: values[-1] for metric, values in metric_values.items()}

    def _save_model(self,
                    output_dir=None,
                    optimizer=None,
                    scheduler=None,
                    model=None,
                    results=None):  # SAVE COMMENTED
        if not output_dir:
            output_dir = self.args.output_dir
        os.makedirs(output_dir, exist_ok=True)

        logger.info(f"Saving model into {output_dir}")
        print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        print(f"Saving model into {output_dir}")

        if model and not self.args.no_save:
            # Take care of distributed/parallel training
            model_to_save = model.module if hasattr(model, "module") else model
            self._save_model_args(output_dir)

            if self.args.model_type in ["bart", "marian"]:
                os.makedirs(os.path.join(output_dir), exist_ok=True)
                model_to_save.save_pretrained(output_dir)
                self.config.save_pretrained(output_dir)
                if self.args.model_type == "bart":
                    self.encoder_tokenizer.save_pretrained(output_dir)
            else:
                os.makedirs(os.path.join(output_dir, "encoder"), exist_ok=True)
                os.makedirs(os.path.join(output_dir, "decoder"), exist_ok=True)
                self.encoder_config.save_pretrained(
                    os.path.join(output_dir, "encoder"))
                self.decoder_config.save_pretrained(
                    os.path.join(output_dir, "decoder"))

                model_to_save = (self.model.encoder.module if hasattr(
                    self.model.encoder, "module") else self.model.encoder)
                model_to_save.save_pretrained(
                    os.path.join(output_dir, "encoder"))

                model_to_save = (self.model.decoder.module if hasattr(
                    self.model.decoder, "module") else self.model.decoder)

                model_to_save.save_pretrained(
                    os.path.join(output_dir, "decoder"))

                self.encoder_tokenizer.save_pretrained(
                    os.path.join(output_dir, "encoder"))
                self.decoder_tokenizer.save_pretrained(
                    os.path.join(output_dir, "decoder"))

            torch.save(self.args, os.path.join(output_dir,
                                               "training_args.bin"))
            if optimizer and scheduler and self.args.save_optimizer_and_scheduler:
                torch.save(optimizer.state_dict(),
                           os.path.join(output_dir, "optimizer.pt"))
                torch.save(scheduler.state_dict(),
                           os.path.join(output_dir, "scheduler.pt"))

        if results:
            output_eval_file = os.path.join(output_dir, "eval_results.txt")
            with open(output_eval_file, "w") as writer:
                for key in sorted(results.keys()):
                    writer.write("{} = {}\n".format(key, str(results[key])))

    def _move_model_to_device(self):
        self.model.to(self.device)

    def _get_inputs_dict(self, batch):
        device = self.device
        if self.args.model_type in ["bart", "marian"]:
            pad_token_id = self.encoder_tokenizer.pad_token_id
            source_ids, source_mask, y = batch["source_ids"], batch[
                "source_mask"], batch["target_ids"]
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone()
            lm_labels[y[:, 1:] == pad_token_id] = -100

            inputs = {
                "input_ids": source_ids.to(device),
                "attention_mask": source_mask.to(device),
                "decoder_input_ids": y_ids.to(device),
                "lm_labels": lm_labels.to(device),
            }
        else:
            lm_labels = batch[1]
            lm_labels_masked = lm_labels.clone()
            lm_labels_masked[lm_labels_masked ==
                             self.decoder_tokenizer.pad_token_id] = -100

            inputs = {
                "input_ids": batch[0].to(device),
                "decoder_input_ids": lm_labels.to(device),
                "labels": lm_labels_masked.to(device),
            }

        return inputs

    def _save_model_args(self, output_dir):  # SAVE COMMENTED
        os.makedirs(output_dir, exist_ok=True)
        self.args.save(output_dir)

    def _load_model_args(self, input_dir):
        args = Seq2SeqArgs()
        args.load(input_dir)
        return args

    def get_named_parameters(self):
        return [n for n, p in self.model.named_parameters()]

In [11]:
## Load and train model

In [12]:
logger = logging.getLogger(__name__)

train_df = pd.read_csv(BASE_FOLDER + BART_INPUT)

test_df = train_df[:10]
test_df = test_df.reset_index()
train_df = train_df[10:20]
train_df = train_df.reset_index()

train_input_ids, train_input_masks, train_input_segment = tokenize_bert(
    train_df["input_text"], tokenizer_intensity)
train_int_labels = intensity_model.predict(
    x=[train_input_ids, train_input_masks]).flatten()

model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 2
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 1
model_args.num_train_epochs = 1
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False

model_args.from_tf = True

model = Seq2SeqModel(encoder_decoder_type="bart",
                     encoder_decoder_name="facebook/bart-base",
                     args=model_args,
                     use_cuda=False,
                     from_tf=True,
                     force_download=True)
model.train_model(train_data=train_df,
                  model_lr=intensity_model,
                  tokenizer_new=tokenizer_intensity,
                  labels=train_int_labels,
                  eval_data = test_df,
                  test_data=test_df)
output.close()

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


in train_model**** eval_data    index  Unnamed: 0                                         input_text  \
0      0           0  What I do know is that I live on a small Islan...   
1      1           1  #BuildThatWall #BuildTheWall #BuildTheDamnWall...   
2      2           2  Couple Befriend Negroid, Negroid Tries To Burn...   
3      3           3  i've felt anger when i've read it on forum, bu...   
4      4           4  Well, the warm weather is coming in and guess ...   
5      5           5  The jew media likes to prop this whites becomi...   
6      6           6  DuetschGirlTX, No I live in a tiny ass town wh...   
7      7           7  I lost several friends over openly supporting ...   
8      8           8  The protesters banged in droves on the glass f...   
9      9           9  Its a bitch who better get her some business f...   

                                         target_text      prefix  
0  What I do know is that I live on a small Islan...  paraphrase  
1  Ask them

  0%|          | 0/10 [00:00<?, ?it/s]

<class 'list'>
ln 10
1 done


...

/usr/local/lib/python3.7/dist-packages/transformers/modeling_bart.py:1075: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


Evaluation loss tensor(2.4476)
Evaluation loss tensor(0.7612)
Evaluation loss tensor(1.6414)
Evaluation loss tensor(1.9167)


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Evaluation loss tensor(1.8258)
complete first kfold


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [13]:
to_predict = [
    str(input_text)
    for input_text in test_df["input_text"].tolist()
]

out=model.predict(to_predict)
print(out)

['What I do know is that I live on a small Island and it seems that a lot of people from various countries in the world are willing to desert their homelands and their families just to come here, in my mind that tells me a lot about said people and their home countries, most of all it highlights their weaknesses of how their own countries have let them down and how they feel the need to feed off somebody elses country, how should we English interpret this without sounding rude???, locust???, parasite???, you tell me. ', "#BuildThatWall #BuildTheWall #buildThatWall TheBuildTheDamnWall # BuildThatWallTheBuildThe DamnWall BuildingThatWall BuildingTheDamn Wall BuildingThat Wall #BuildThat Wall BuildingThe Damn Wall Building That Wall BuildThat Wall TheBuild ThatWall BuildTheWall BUILD THAT EFFEN WALL! I'm sorry my Lord #Jesus but people are just deaf down here", 'Couple Befriend Negroid, Negroid Tries To Burn Down Couples Home - NNN Reporters Newsroom Forum Moblie Couple Says Neighbor Trie

In [ ]:
# model.save_pretrained(BASE_FOLDER + OUTPUT_FOLDER + OUTPUT_FILE)